In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import keras
import tensorflow as tf
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.metrics import accuracy_score
from google.colab import files

### Load Dataset

In [70]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#load data from csv
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Innolab/test.csv")
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Innolab/train.csv")

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/Innolab")
import data_loader as dtl
# import nn_modelling_utils as ut

### Preprocessing and preperation for CNN

In [71]:
test.head(2)

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,1_10,1_11,1_12,1_13,1_14,1_15,1_16,1_17,1_18,1_19,1_20,1_21,1_22,1_23,1_24,1_25,1_26,1_27,1_28,1_29,1_30,...,237_243,237_244,237_245,237_246,238_239,238_240,238_241,238_242,238_243,238_244,238_245,238_246,239_240,239_241,239_242,239_243,239_244,239_245,239_246,240_241,240_242,240_243,240_244,240_245,240_246,241_242,241_243,241_244,241_245,241_246,242_243,242_244,242_245,242_246,243_244,243_245,243_246,244_245,244_246,245_246
0,249,46c3433d4,17,74,26.03.2018,1,2,13,-2.049570,1.0,249.0,0.574942,0.305049,0.162847,0.275940,0.019306,-0.019452,-0.156510,0.172117,-0.024951,0.606818,0.441176,0.084196,0.114680,0.521914,0.198896,0.475564,0.192633,0.307193,0.149679,0.464812,0.002804,0.411915,0.287092,0.335296,0.148400,0.475040,0.083054,0.172027,-0.042041,...,0.361282,0.681434,0.051703,0.356248,0.021860,0.151483,0.046508,0.380081,0.190855,0.532496,0.067998,0.159488,0.267005,0.247733,-0.007360,0.139081,0.111605,0.154606,0.293677,0.355418,0.622377,0.157162,0.485685,0.207092,0.651844,0.124603,0.476194,0.08167,-0.140328,0.017211,0.017766,0.713473,0.160039,0.407091,0.193513,-0.086803,0.171596,0.276317,0.494943,0.344752
1,577,b18d729af,11,76,15.06.2016,0,2,11,-1.189132,1.0,577.0,0.776586,0.467788,0.164115,0.321399,0.203079,0.344261,-0.015906,0.436849,0.470886,0.626489,0.487020,0.106208,-0.003195,0.091799,0.050809,0.650164,0.355355,0.123783,-0.136501,0.252601,0.134610,0.301837,0.111609,0.667055,0.223697,-0.085763,-0.107308,0.598378,0.452597,...,0.525016,0.778868,0.368278,0.479114,0.447165,0.711927,0.669327,0.576660,0.367612,0.577063,0.239827,0.515898,0.203085,0.514212,0.144925,0.366177,0.213815,0.856538,0.582982,0.621164,0.785972,0.603620,0.814200,0.049528,0.382625,0.495069,0.316740,0.37590,0.209996,0.402320,0.464287,0.413642,-0.051788,0.284676,0.837725,0.315466,0.441045,0.197800,0.410294,0.692895


In [72]:
scaler = StandardScaler()

In [73]:
def preprocess_conn_matrix(df, train_data = True):
    #create target
    y, X = dtl.preprocess_data(df)
    y = np.array(y, dtype=np.float32)

    #drop columns
    cols_to_drop = ["MEM_score", "Apoe", "age", "sex", "edyears"]
    X.drop(columns=cols_to_drop, inplace = True)
    
    # if train_data:
    #     scaler.fit_transform(X)
    # else:
    #     scaler.transform(X)

    #turn array to matrix
    n_c = dtl.flat_to_mat(X.iloc[0,:]).shape[0]
    n_train = len(X)
    X_train_2d = np.zeros(n_train*n_c*n_c).reshape(n_train,n_c,n_c)

    for i in range(n_train):
      X_train_2d[i] = dtl.flat_to_mat(X.iloc[i,:])

    #reshape data
    X_train_2d = X_train_2d.reshape(X_train_2d.shape[0], 1, X_train_2d.shape[1], X_train_2d.shape[2])

    return X_train_2d, y

In [ ]:
def data_augmentation_gaussian(X,y, scale = .17):
    X_noise = X.copy()
    y_noise = y.copy()

    for i in range(X.shape[0]):
        #add gaussian noise
        noise = np.random.normal(loc = 0, scale = scale, size = (X.shape[1],X.shape[2])
        X_noise[i] = X[i] + np.triu(noise, k=1) + np.triu(noise, k=1).T

    return np.concatenate((X, X_noise), axis = 0), np.concatenate((y, y_noise), axis = 0)

In [ ]:
X_train, y_train = preprocess_conn_matrix(train, train_data = True)
X_test, y_test = preprocess_conn_matrix(test, train_data = False)

In [ ]:
X_train,y_train = data_augmentation_gaussian(X_train,y_train, scale = .17)

In [ ]:
## TODO: Implement CCNN https://www.researchgate.net/publication/318652568_Resting_State_fMRI_Functional_Connectivity-Based_Classification_Using_a_Convolutional_Neural_Network_Architecture/link/5978a095aca27203ecc4a2a1/download
## Write training routine
# validation batch
# add visualization